In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 3

In [3]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [4]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 3
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.304669  [   64/60000]
loss: 2.302788  [ 6464/60000]
loss: 2.280457  [12864/60000]
loss: 2.275224  [19264/60000]
loss: 2.253619  [25664/60000]
loss: 2.209328  [32064/60000]
loss: 2.233925  [38464/60000]
loss: 2.188278  [44864/60000]
loss: 2.187278  [51264/60000]
loss: 2.155161  [57664/60000]
Test Error: 
 Accuracy: 39.1%, Avg loss: 2.153430 

Epoch 2
-------------------------------
loss: 2.156367  [   64/60000]
loss: 2.159839  [ 6464/60000]
loss: 2.094632  [12864/60000]
loss: 2.116324  [19264/60000]
loss: 2.068507  [25664/60000]
loss: 1.986702  [32064/60000]
loss: 2.042479  [38464/60000]
loss: 1.944876  [44864/60000]
loss: 1.954993  [51264/60000]
loss: 1.889238  [57664/60000]
Test Error: 
 Accuracy: 59.0%, Avg loss: 1.885964 

Epoch 3
-------------------------------
loss: 1.911224  [   64/60000]
loss: 1.894739  [ 6464/60000]
loss: 1.764097  [12864/60000]
loss: 1.813634  [19264/60000]
loss: 1.708426  [25664/60000]
loss: 1.636710  [32064/600

In [5]:
with torch.no_grad():
    for param in model.parameters():
        print(param.shape, param.numel())

torch.Size([512, 784]) 401408
torch.Size([512]) 512
torch.Size([512, 512]) 262144
torch.Size([512]) 512
torch.Size([10, 512]) 5120
torch.Size([10]) 10
